<a target="_blank" href="https://colab.research.google.com/github/sonder-art/mhar/blob/released/nbs/tutorial_nonfull_dimensional.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

Uncomment the next snippet if you are in colab or need to install mhar library!

In [ ]:
#!pip install mhar

This tutorial will show how to use `mhar` for sampling non-full dimensional polytopes. It is focused on executing parallel MCMC walks over a polytope in GPUs. If you want to se a tutorial on how to sample full dimensional polytopes see [tutorial](https://github.com/sonder-art/mhar/blob/released/nbs/tutorial_full_dimensional.ipynb)

In [1]:
import torch

Before starting let's check if you have an avaialble gpu device or not.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print('Device:', device)

Device: cuda


We also need to decide the data-type `dtype` we are going to use. Depending on your necessities you can choose it, we recomend to use `64` bits for non-fully dimentional polytopes in order to maintain numerical inestability of the projections. Otherwiise the precision depends on the dimension of your polytope and speed you want.  
  
As of now `16` bit precision is only available for `gpu` and not `cpu`.

In [3]:
# We will choose 64-bits
dtype = torch.float64

## Canonical Representation

The polytope in question must be presented in matrix canonical representation (as opposed to vertex). `mhar` assumes that the matrix has no repeated or redundant restrictions.

## Non-Fully dimensional Polytopes  

### Definition

> $A^IX \leq b^I$  
> 
> $A^EX = b^E$

For non-fully dimensional polytopes we need to use the class [`NFDPolytope`](https://sonder-art.github.io/mhar/polytope.html#nfdpolytope) in the `mhar.polytope` module. The restrictions must be passed as pytorch tensors.  
  
We will sample the unit hypercube that is defined as:  
> $n-simplex = \{x \in R^n || \sum_{i=1}^{n} x_i = 1, 0 \leq x_i \} $  

Which we can represent in matrix restrictions:  
$ -Ix \leq 0$  
$ [1]^n = 1 $  
Where $I$ is the identity matrix of dimension $n \times n$ 

#### Definition-Code

Lets create the tensors to represent the restrictions that define the polytope. Since we need to create a projection matrix for the non-fully dimensional object we need to preserve the numerical stability of the algorithm, we suggest using 64 bits precision. 

In [4]:
n = 3 # Dimension
dtype = torch.float64 # Precision 
A_I = torch.eye(n).to(dtype) * -1.0
b_I = torch.empty(n, 1, dtype=dtype)
b_I.fill_(0.0)

# Create Equalities
A_E = torch.empty(1, n, dtype=dtype)
A_E.fill_(1.0)
b_E = torch.empty(1, 1, dtype=dtype)
b_E.fill_(1.0)      
print(f'Inequality Matrix A^I \n {A_I} \n')
print(f'Inequality Vector b^I \n {b_I}\n')
print(f'Equality Matrix A^E \n {A_E} \n')
print(f'Equality Vector b^E \n {b_E}')

Inequality Matrix A^I 
 tensor([[-1., -0., -0.],
        [-0., -1., -0.],
        [-0., -0., -1.]], dtype=torch.float64) 

Inequality Vector b^I 
 tensor([[0.],
        [0.],
        [0.]], dtype=torch.float64)

Equality Matrix A^E 
 tensor([[1., 1., 1.]], dtype=torch.float64) 

Equality Vector b^E 
 tensor([[1.]], dtype=torch.float64)


Now lets create a [`NFDPolytope`](https://sonder-art.github.io/mhar/polytope.html#nfdpolytope) object to represent the polytope.

In [5]:
from mhar.polytope import NFDPolytope
simplex = NFDPolytope(A_I, # Inequality Restriction Matrix 
                     b_I,  # Inequality Vector
                     A_E, # Equality Restriction Matrix 
                     b_E,  # Equality Vector
                     dtype, # torch dtype
                     device, # device used cpu or cuda
                     copy=False # bool for creating a copy of the restrictions
                     )

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



In [6]:
simplex

Numeric Precision (dtype) torch.float64
Device: cuda
A_in: torch.Size([3, 3]) 
b_in: torch.Size([3, 1])
A_eq: torch.Size([1, 3]) 
b_eq: torch.Size([1, 1])

### Projection Matrix

Now we need to compute que projection matrix that we will use for projecting the random directions vectors to the equality space. For that we can use the method `NFDPolytope.compute_projection_matrix()`. We recommend using the highest precision possible to compute this matrix.

In [7]:
simplex.compute_projection_matrix(device=device, solver_precision=torch.float64)

Max non zero error for term (A A')^(-1)A at precision torch.float64:  tensor(0., device='cuda:0', dtype=torch.float64)


In [8]:
simplex

Numeric Precision (dtype) torch.float64
Device: cuda
A_in: torch.Size([3, 3]) 
b_in: torch.Size([3, 1])
A_eq: torch.Size([1, 3]) 
b_eq: torch.Size([1, 1])
Projection Matrix: torch.Size([3, 3])

### Starting Inner Point(s)

In order to start the algorithm we need at least one inner point $x_0$. If you know your inner point you can supply it to the algorithm, `mhar` also contains functions to compute one inner point using the [chebyshev center](https://en.wikipedia.org/wiki/Chebyshev_center) which finds the center of the smallest ball inside the polytope.

 `from mhar.inner_point import ChebyshevCenter`. The solver is in numpy so precision must be specified as `numpy.dtype`. It uses `linprog` from `scipy.optimize`. You can see the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html). 
  
It could also be the last points produced by a previous walk/run of the `mhar` 

In [9]:
from mhar.inner_point import ChebyshevCenter
import numpy as np

In [10]:
x0 = ChebyshevCenter(polytope=simplex, # Polytope Object
                    lb=None,  # Lowerbound (lb <= x ), if unknown leave it as None 
                    ub=None,  # Upperbound ( x <= up), if unknown leave it as None 
                    tolerance=1e-4, # Tolerance for equality restrictions (A_eqx = b_eq)
                    device=device, # device used cpu or cuda
                    solver_precision=np.float64 # numpy dtype
                    )


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


In [11]:
x0

tensor([[0.3333],
        [0.3333],
        [0.3333]], device='cuda:0', dtype=torch.float64)

If we want to manually input the inner points then it is enough to use a torch tensor of size $n \times l$. Where $l$ is ne number of inner points you want to supply. Just write them in column notation.  
  
We are going to manually add an other starting point to the one calcualted by the `chebyshev center` to show its functionality later.

In [12]:
x0 = torch.cat([x0, 
            torch.tensor([[.25], [.25], [.5]]).to(device).to(dtype)
             ], dim=1)
x0

tensor([[0.3333, 0.2500],
        [0.3333, 0.2500],
        [0.3333, 0.5000]], device='cuda:0', dtype=torch.float64)

Now we can proceed to sample the `polytope`

### Walk

We are going to sample the polytope starting from the inner points we supply using the method `walk.walk`. It has the next arguments:

+ `polytope` is an object of the type [`Polytope`](https://sonder-art.github.io/mhar/polytope.html#polytope) or [`NFDPolytope`](https://sonder-art.github.io/mhar/polytope.html#nfdpolytope) that defines it.
+ `X0` a tensor containing the inner points to start the walks from.
+ `z` determines the number of simoultaneous `walks`. If the number of initial points supplied are less than `z`  ($ncols($ `x0` $) < $ `z`) then some points will be reused as starting points.  
+ `T` is the number of uncorrelated iterations you want. The number of total uncorrelated points produced by the algorithm is `z` $\times$ `T`, since `z` points are sampled at each iteration.  
+ `thinning` determines the number of points that we need to burn between iterations in order to get uncorrelated points. The suggested factor should be in the order of $O(n^3)$.
+ `warm` determines a thinning for warming the walks only at the beggining, after the this war the walks resumes as normal. It is used if you want to lose the dependency from the starting points.
+ `device` device where the tenros live `cpu` or `cuda`
+ `seed` for reproducibility
+ `verbosity` for printing what is going on

In [13]:
x0

tensor([[0.3333, 0.2500],
        [0.3333, 0.2500],
        [0.3333, 0.5000]], device='cuda:0', dtype=torch.float64)

In [14]:
from mhar.walk import walk
X = walk(polytope=simplex,
        X0 = x0,  
        z=100, 
        T=10, 
        warm=0,
        thinning=n**3, 
        device=device, 
        seed=None,
        verbosity=2
)

Minimum number allowed -1.7976931348623157e+308
Maximum number allowed 1.7976931348623157e+308
Eps:  2.220446049250313e-16
Values close to zero will be converted to 3eps or -3eps: 6.661338147750939e-16
n:  3   mI: 3   mE: 1   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |███---------------------------| 10.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████------------------------| 20.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |█████████---------------------| 30.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |████████████------------------| 40.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |███████████████---------------| 50.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████------------| 60.0%
% of burned samples |██████████████████████████████| 100.0%
% of iid sa

[`walk`](https://sonder-art.github.io/mhar/walk.html#walk) produces `T` $\times$ `z` uncorrelated points. It returns a vector of dimension `T` $\times$ `z` $\times$ `n`.  

In [15]:
X

tensor([[[0.0889, 0.0212, 0.4025,  ..., 0.5306, 0.0667, 0.5547],
         [0.1682, 0.8578, 0.4345,  ..., 0.4050, 0.1121, 0.3566],
         [0.7429, 0.1210, 0.1629,  ..., 0.0644, 0.8212, 0.0886]],

        [[0.7528, 0.1165, 0.4401,  ..., 0.1687, 0.6633, 0.1609],
         [0.1467, 0.6279, 0.3744,  ..., 0.6327, 0.1527, 0.7232],
         [0.1004, 0.2556, 0.1854,  ..., 0.1986, 0.1840, 0.1159]],

        [[0.0813, 0.0936, 0.2726,  ..., 0.1242, 0.0878, 0.2947],
         [0.5696, 0.6319, 0.7036,  ..., 0.0086, 0.4037, 0.5547],
         [0.3492, 0.2745, 0.0238,  ..., 0.8672, 0.5085, 0.1505]],

        ...,

        [[0.6753, 0.3518, 0.0888,  ..., 0.6357, 0.7667, 0.1516],
         [0.3004, 0.6083, 0.0235,  ..., 0.1452, 0.0366, 0.5145],
         [0.0244, 0.0399, 0.8877,  ..., 0.2191, 0.1967, 0.3339]],

        [[0.0865, 0.2683, 0.0859,  ..., 0.5572, 0.1254, 0.4516],
         [0.7311, 0.5322, 0.0796,  ..., 0.0588, 0.0967, 0.2368],
         [0.1824, 0.1995, 0.8346,  ..., 0.3841, 0.7779, 0.3116]],

 

In [16]:
X.shape

torch.Size([10, 3, 100])

In [17]:
X.sum(1)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1

In [18]:
X.shape

torch.Size([10, 3, 100])

### Numerical Stability

There is some numerical inestability that causes the algorithm to degrade overtime, we recommend checking your walk everyonce in a while and be sure that it is not a big deal. The inestability is due to the projection matrix.

In [19]:
tol = 1e-10
print('Infinite: ', (~torch.isfinite(X)).sum())
print('Nans:  ',(torch.isnan(X)).sum())
print('Inequality violation:  ',(X.sum(1)!=1.0).sum())
print(f'Inequality violation with tol {tol}:  ',((X.sum(1)- 1.0).abs()>tol).sum())

Infinite:  tensor(0)
Nans:   tensor(0)
Inequality violation:   tensor(927)
Inequality violation with tol 1e-10:   tensor(0)


### Summary

To sumamrize the steps taken we can use the `polytope_examples` for creating a [`Hypercube`](https://sonder-art.github.io/mhar/polytope_examples.html#hypercube).

In [20]:
from mhar.polytope_examples import Simplex

# Create a polytope (Simplex)
simplex_sim = Simplex(10,
                      dtype=torch.float64,
                      device=device
                      )

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



In [21]:
simplex_sim.compute_projection_matrix(device=device)

Max non zero error for term (A A')^(-1)A at precision torch.float64:  tensor(2.2204e-16, device='cuda:0', dtype=torch.float64)


Define/Find inner points

In [22]:
x0_sim = ChebyshevCenter(polytope=simplex_sim, 
                    lb=None, 
                    ub=None, 
                    tolerance=1e-4,
                    device=device,
                    solver_precision=np.float64)


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


Sample points

In [23]:
X_sim = walk(polytope=simplex_sim,
        X0 = x0_sim,  
        z=100, 
        T=1, 
        warm=0,
        thinning=10000, 
        device= None, 
        seed=None,
        verbosity=2
)
X_sim

Minimum number allowed -1.7976931348623157e+308
Maximum number allowed 1.7976931348623157e+308
Eps:  2.220446049250313e-16
Values close to zero will be converted to 3eps or -3eps: 6.661338147750939e-16
n:  10   mI: 10   mE: 1   z: 100


% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


tensor([[[1.1520e-01, 1.9804e-02, 7.7763e-02, 9.9372e-02, 1.0666e-01,
          4.7353e-02, 2.4089e-02, 1.2750e-01, 3.4367e-02, 4.7841e-02,
          6.7679e-02, 2.0295e-02, 9.1836e-02, 1.6823e-02, 1.8291e-02,
          7.0606e-02, 5.0651e-02, 3.5442e-02, 1.4869e-03, 3.1108e-02,
          4.8304e-02, 6.2594e-02, 2.1732e-01, 3.4688e-02, 5.0051e-02,
          1.2728e-02, 1.1719e-01, 2.2064e-03, 1.3654e-02, 1.7988e-02,
          5.2856e-02, 1.3055e-02, 6.6507e-02, 7.4944e-03, 2.8870e-02,
          3.0513e-01, 2.7739e-02, 1.1810e-01, 1.0644e-01, 5.5130e-02,
          1.9186e-01, 4.1276e-03, 2.2309e-01, 7.0959e-02, 1.5875e-02,
          1.5771e-01, 7.4498e-03, 6.6981e-02, 9.2846e-02, 1.2573e-01,
          9.0559e-02, 2.9251e-02, 3.5833e-01, 1.1234e-01, 1.9873e-01,
          2.3755e-02, 7.5568e-04, 2.1011e-01, 6.4324e-03, 2.0992e-02,
          1.4870e-01, 1.4210e-01, 2.6001e-02, 1.6850e-01, 9.5598e-03,
          7.8793e-02, 1.8993e-01, 2.3360e-01, 3.3954e-01, 5.1940e-02,
          4.8236e-01

In [24]:
X_sim.shape

torch.Size([1, 10, 100])

In [25]:
tol = 1e-10
print('Infinite: ', (~torch.isfinite(X)).sum())
print('Nans:  ',(torch.isnan(X)).sum())
print('Inequality violation:  ',(X.sum(1)!=1.0).sum())
print(f'Inequality violation with tol {tol}:  ',((X.sum(1)- 1.0).abs()>tol).sum())

Infinite:  tensor(0)
Nans:   tensor(0)
Inequality violation:   tensor(927)
Inequality violation with tol 1e-10:   tensor(0)
